In [89]:
from NCRAttachments import NCRAttachments
from functools import lru_cache
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
import json
import requests

In [153]:
CACHE_EXPIRY = 3600*24  # 1 day

@lru_cache(maxsize=1)
def get_cached_secrets(timestamp,secret_names=()):
    # This function will only be called when the timestamp changes
    # Implement your chosen method of secret retrieval here
    # For this example, we'll use AWS Systems Manager Parameter Store
    import boto3
    session = boto3.Session(region_name='us-east-2')
    ssm = session.client('ssm')
    response = ssm.get_parameters(
        Names=secret_names,
        WithDecryption=True
    )
    return {param['Name']: param['Value'] for param in response['Parameters']}

def get_secrets(secret_names):
    # Round the current timestamp to the nearest hour
    current_time = int(time.time() / CACHE_EXPIRY) * CACHE_EXPIRY
    return get_cached_secrets(current_time,secret_names=secret_names)

secrets = get_secrets(secret_names=('SHAREPOINT_TENANT_ID','SHAREPOINT_CLIENT_ID','SHAREPOINT_CLIENT_SECRET'))
#print(secrets)

In [12]:
credentials={
            'tenant_name_or_id': secrets.get('SHAREPOINT_TENANT_ID'),
            'client_id': secrets.get('SHAREPOINT_CLIENT_ID'),
            'client_secret': secrets.get('SHAREPOINT_CLIENT_SECRET'),
    }
ncr_attachments = NCRAttachments(credentials)


In [ ]:
ncrs = await ncr_attachments.list_all_NCRs()

In [ ]:
len(ncrs)

In [ ]:
ncrs[0].fields.__dict__

In [ ]:
from microsoft_graph_accessor import MicrosoftGraphAccessor
#attachments
mga = MicrosoftGraphAccessor(credentials)
##find site assets
root = mga.get_root()
sites = mga.get_drive_list_from_site(ncr_attachments.qc_site)
[site.name for site in sites]

In [ ]:
from msgraph.generated.sites.sites_request_builder import SitesRequestBuilder
from kiota_abstractions.base_request_configuration import RequestConfiguration
# To initialize your graph_client, see https://learn.microsoft.com/en-us/graph/sdks/create-client?from=snippets&tabs=python
query_params = SitesRequestBuilder.SitesRequestBuilderGetQueryParameters(
		select = ["siteCollection","webUrl"],
		filter = "siteCollection/root ne null",
)

request_configuration = RequestConfiguration(
query_parameters = query_params,
)
await mga.msgraph_client.sites.get(request_configuration=request_configuration)

In [ ]:
site_id = ncr_attachments.qc_site.id
list_id = ncr_attachments.ncr_list.id
item_id=2
attachments_response = await mga.msgraph_client.sites.by_site_id(site_id).lists.by_list_id(list_id).items.by_list_item_id(item_id).get()
attachments_response
# attachments = attachments_response.json().get("value", [])

# for attachment in attachments:
#     attachment_url = attachment.get("contentUrl")
#     print(f"Attachment Name: {attachment['name']}, URL: {attachment_url}")

In [ ]:

headers = {
    "Authorization": f"Bearer {sharepoint_access_token}",
    "Accept": "application/json;odata=verbose"
}
response = requests.get(url,headers=headers)
print(response.__dict__)

In [ ]:
print(ncr_attachments.ncr_list.id)
await mga.msgraph_client.drives.by_drive_id().items.by_drive_item_id(2).get()

In [ ]:
token

In [154]:

#json.loads(sharepoint_api_token.content)

In [ ]:
import webbrowser
tenant_id = secrets.get('SHAREPOINT_TENANT_ID')
client_id = secrets.get('SHAREPOINT_CLIENT_ID')
client_secret = secrets.get('SHAREPOINT_CLIENT_SECRET')
redirect_uri = 'https://login.microsoftonline.com/common/oauth2/nativeclient'
auth_url = f'''https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/authorize
?client_id={client_id}
&client_secret={client_secret}
&response_type=code
&redirect_uri={redirect_uri}
&response_mode=query
&scope=https://iscopipe.sharepoint.com/.default offline_access
&state=12345'''

print(auth_url)

In [146]:
auth_code = '0.ARsA9eVuJpHbM0GmLNt63lMC86QeLxO63ktLqk-k6t1QKTUbAEw.AgABBAIAAADW6jl31mB3T7ugrWTT8pFeAwDs_wUA9P_iF9WrfSwuuiDYKf9k8vYMC822Fq6h-OkY_GZhWPMAI4rQofvwXvmcxA-_g163olt_9FelIC0IlqN0j9B4BA003LYMmMj9x8c-Hiir9298Ykc7Xh-VW4vZVHDqK8RY56cAGoRGp_tnPfaA1_VPpp4I_uSZEukkKs0-DianCOMcxpYuTnCFuSsIQrkN8909n7P4h7TITg7B6PWUvSNnSyliE2k3ifOQjl9nD9pFnGNsiD20Y-X0RNsZwD6_VFunMhnRxrr46FI1prgGAEB6zCtkYvNc9Tc2xpBV3zivCHLuAJF1VJkHDrWyzmmUXuZ9hCdNYggIEuPmQXErrn4LolVj0QtyP2oFpj5YTw-Soiy5Z3SB01VbmpAq-3wvU2XQpkFWNF4HksEY2J0vXRnyp3o74QnjmxdknKeCD2QflnRIcDa1z24NloUzvUaACpeD8Lu1qwC3L5ju8hLGU3bqGf5bXyfPTzxbBI0inAZexp_1ce_Sk3kU1GvQqWTFJ9L0P19T8_x4p5UG18C8viE3BCwLUEMakhY2qMR1xMZYwFlF-C_cAamDzAaCbbBHN7lnE9wK7N4w34d_YhSBsHBA56so4_ewZLUgMaVcq7mrN0hOqtWCRPNvLV9NFYieJDSnMGwhPkadGFdOpE-MRm2njwwnm12ARwXR3aeXMHVK3uaF7ark69Vg82g_uO0WM5PDvcDW87wJEnMhbDIJSrV1hr_WtDdJ4f7JF-oNzYjAGAPJX0SpJkDjtBnCH4FjN5jZs1WcsdkKpheOJTxW0XcC4Qk8Mcte3xtqLvqhL_U3fX6vm0IZH8HKEJqxxmdd0KNey7Jr3Q9ePVWE2QkeQvLSNxwRhCxk8qtYGY-yY6stI_aGd66sEzF7oNwEi7BsTArboVQPXa3v'

In [ ]:
import requests
import json

# Get the token via the previously established process
#token = mga.client.__dict__['_acquire_token_callback']()
redirect_uri = "https://login.microsoftonline.com/common/oauth2/nativeclient"
# Request data for getting access token
request_data = {
    'code': auth_code,
    'client_id': secrets.get('SHAREPOINT_CLIENT_ID'),
    'client_secret': secrets.get('SHAREPOINT_CLIENT_SECRET'),
    'grant_type': 'authorization_code',
    'scope': 'https://iscopipe.sharepoint.com/.default offline_access',
    'redirect_uri': redirect_uri
}

tenant_id = secrets.get('SHAREPOINT_TENANT_ID')

# Request to get the SharePoint API token
sharepoint_api_token_request = requests.post(
    f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token',
    data=request_data
)

sharepoint_api_token_request_content = json.loads(sharepoint_api_token_request.content)
print(sharepoint_api_token_request_content)
#Extract access token
sharepoint_access_token = sharepoint_api_token_request_content['access_token']
#
# Verify the token length
print(f"Access token length: {len(sharepoint_access_token)}")

# Construct the SharePoint API URL
tenant_id = 'iscopipe'
list_name = 'NCR Log'
item_id = '2'

# Notice the corrected usage of double and single quotes in the URL
url = f"https://{tenant_id}.sharepoint.com/QC/_api/web/lists(guid'{ncr_attachments.ncr_list.id}')/items/GetById({item_id})/AttachmentFiles"

#url = "https://iscopipe.sharepoint.com/QC/Lists/NCR%20Log/Attachments/2/"

print(f"Generated URL: {url}")

# Headers for the GET request to access attachment files
headers = {
    "Authorization": f"Bearer {sharepoint_access_token}",
    "Accept": "application/json;odata=verbose"
}

# Make the GET request to fetch the attachment files
response = requests.get(url, headers=headers)

# Print the response or handle errors
if response.status_code == 200:
    print(f"Response: {response.json()}")
else:
    print(f"Error: {response.status_code}, {response.text}")


In [ ]:
headers = {
    "Authorization": f"Bearer {sharepoint_access_token}",
    "Accept": "application/json;odata=verbose"
}

url = "https://iscopipe.sharepoint.com/QC/_api/Web/Lists(guid'c5325d9e-516d-4564-a8fa-18427ffe4ea7')/Items(2)/AttachmentFiles('Republic%20Services%20Perf%20Pattern.pdf')/$value"

response = requests.get(url, headers=headers)
print(response.content)
# Check if the request was successful
if response.status_code == 200:
    # Open a local file in write-binary mode to save the content
    with open('Republic%20Services%20Perf%20Pattern.pdf', 'wb') as file:
        file.write(response.content)  # Write the file content to disk
    print("File downloaded successfully.")
else:
    # Handle the error
    print(f"Error: {response.status_code}, {response.text}")